In [ ]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import requests


In [ ]:
response = requests.get('https://datos.madrid.es/egob/GET/catalogo/209434-0-templos-otros.json')
jsons_1 = response.json()
jsons_1.keys()
json_trabajado = jsons_1['@graph']

In [ ]:
df_flat_1 = pd.json_normalize(json_trabajado)

In [ ]:
responses = requests.get('https://datos.madrid.es/egob/GET/catalogo/209426-0-templos-catolicas.json')
json = responses.json()

In [ ]:
json_trabajado2 = json['@graph']

In [ ]:
df_flat = pd.json_normalize(json_trabajado2)

In [ ]:
df_templos = pd.concat([df_flat, df_flat_1], axis=0)

In [ ]:
df_templos.tail(10)

In [ ]:
templos_df = df_templos.drop(['@id', '@type', 'id','relation','address.district.@id', 'address.area.@id', 'address.locality', 'address.postal-code','organization.organization-desc','organization.accesibility', 'organization.schedule', 'organization.services', 'organization.organization-name'], axis =1)

In [ ]:
templos_df

In [ ]:
connect_str = 'mysql+pymysql://ironhack_user:%Vq=c>G5@173.201.189.217/BiciMAD'
engine = create_engine(connect_str)
inspector = inspect(engine)

In [ ]:
query_1 = '''

SELECT *
FROM bicimad_stations
'''

df = pd.read_sql_query(query_1, connect_str)

In [ ]:
df[["longitud","latitud"]]=df["geometry.coordinates"].str.split(",",expand=True)
df['longitud'] = df['longitud'].str.replace('[','',regex=True)
df["latitud"]=df['latitud'].str.replace(']','',regex=True)

df['latitud'] = df['latitud'].astype(float)
df['longitud'] = df['longitud'].astype(float)
df

In [ ]:
estaciones_df = df.drop(['id', 'light', 'number','activate','no_available','geometry.coordinates', 'total_bases', 'dock_bikes', 'free_bases','reservations_count','geometry.type'], axis =1)

In [ ]:
estaciones_df

In [ ]:
union = templos_df.assign(key="union").merge(estaciones_df.assign(key="union"), how='left', on='key')
union

In [ ]:
longitud_bicis = list(estaciones_df['longitud'])

In [ ]:
latitud_bicis = list(estaciones_df['latitud'])

In [ ]:
longitud_templos = list(templos_df['location.longitude'])

In [ ]:
latitud_templos = list(templos_df['location.latitude'])

In [ ]:
from shapely.geometry import Point
import geopandas as gpd   # conda install -c conda-forge geopandas

def to_mercator(lat, long):
    # transform latitude/longitude data in degrees to pseudo-mercator coordinates in metres
    c = gpd.GeoSeries([Point(lat, long)], crs=4326)
    c = c.to_crs(3857)
    return c

In [ ]:
def distance_meters(lat_start, long_start, lat_finish, long_finish):
    # return the distance in metres between to latitude/longitude pair point in degrees (i.e.: 40.392436 / -3.6994487)
    start = to_mercator(lat_start, long_start)
    finish = to_mercator(lat_finish, long_finish)
    return start.distance(finish)

In [ ]:
union['p'] = union.apply(lambda x: distance_meters(40.414203, -3.709672, x['latitud'], x['longitud']),axis=1)


In [ ]:
union